# [DIS 4] Principles of CNN
The original contributor: Suhong Moon

**If you are running on a local anaconda install, you will need to install pytorch with the command**
```sh
conda install pytorch -c pytorch
```

Many materials are from Deep Learning book (Ian Goodfellow, Yoshua Bengio and Aaron Courville)

Why does CNN perform better than MLP (Multilayer Perceptron) in various modalities? In this notebook, you will understand the principle of CNN through various examples and experiments. The three most distinct features that differentiate CNN from MLP are as follows:

1. **sparse interactions** : Unlike the MLP model, which had to calculate the interactions between all neurons using matrix multiplication, CNN has sparse interactions. This is achieved by using smaller kernels in comparison to the resolution of the input image. This means that CNN can greatly reduce the amount of computation and memory requirements and improve statistical efficiency. This is also called sparse connectivity or sparse weights.


2. **parameter sharing** : Parameter sharing means using the same parameters more than once within a model. In the case of MLP, all parameters are used only once when calculating the output within one layer. This reduces the memory used to store parameters.


3. **translational equivariance** : Parameter sharing in convolution operation makes the convolution layer equivariant to translation of given input. When a function is equivariant to some operation, it means that when the input changes as much as the given operation, the output of the function also changes in the same way. To explain it more formally, if a function $f(x)$ is equivariant to a transformation $g(x)$, then $f(g(x)) = g(f(x))$. In the case of convolution, $g(x)$ is the translation of the input $x$. 

    While convolution is equivariant to translation, it is not equivariant to other transformations such as rotation, scale, or warping. Therefore, various regularizations such as data augmentations are used to obtain CNN functions that are robust to such transformations during training. However, this will not be covered in this notebook.



In [ ]:
# As usual, a bit of setup

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import matplotlib.pyplot as plt
import numpy as np
import random 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import IPython
from ipywidgets import interactive, widgets, Layout
from IPython.display import display, HTML

seed = 7
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## MLP vs CNN

In this problem, we will learn why the properties that CNN have lead to better performance on the vision modality by comparing it with MLP. 

The number of parameters is the rule of thumb to compare the expressiveness of the neural network. So we are now comparing MLP and CNN that have the similar number of parameters. Let's see how the performance deviates. 

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

PyTorch provides convenient tools to automate the process of downloading 
common datasets, processing the data, and splitting into minibatches.

In [ ]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
# If we want to add data augmentations, torchvision also offers different 
# transformations that we can compose in here, though we would need to be sure
# to have two sets of transformations: one with data augmentation for the 
# training loaders, and one without for the test and validation sets.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./deeplearning/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./deeplearning/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./deeplearning/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

You have an option to **use GPU by setting the flag to True below**. It is recommended, but not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment.

## Colab Users

If you are using Colab, you need to manually switch to a GPU device. You can do this by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`. Note that you have to rerun the cells from the top since the kernel gets restarted upon switching runtimes.

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 500

print('using device:', device)

### Training Helpers

In [ ]:
def check_valid_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

def train_model(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API and prints model 
    accuracies during training.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Lists of validation accuracies at the end of each epoch.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    train_accs = []
    val_accs = []
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each trainable parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Epoch [%d/%d]: Iteration %d, loss = %.4f' % (e, epochs, t, loss.item()))
                check_valid_accuracy(loader_val, model)
                print()
        val_accs.append(check_valid_accuracy(loader_val, model))
    return val_accs

### Define 3 Layer MLP and CNN

In [ ]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, channel_1, 5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(channel_1, channel_2, 3, stride=1, padding=1)
        self.classifier = nn.Linear(channel_2 * 32 * 32, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.flatten(start_dim=1)
        x = self.classifier(x)

        return x

class ThreeLayerMLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x=  F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.classifier(x)

        return x

### Count the number of parameters

Note that with the given architecture hyperparameters, the number of parameters of MLP model is slightly larger than that of CNN.

In [ ]:
def count_parameters(model):
    num_params = sum(p.numel() for p in model.parameters())
    return num_params

input_size = 3 * 32 * 32
in_channel = 3
channel_1 = 100
channel_2 = 100
num_classes = 10
hidden_size = 350

mlp_model = ThreeLayerMLP(input_size, hidden_size, num_classes)
cnn_model = ThreeLayerConvNet(in_channel, channel_1, channel_2, num_classes)

num_params_mlp = count_parameters(mlp_model)
num_params_cnn = count_parameters(cnn_model)

print(f"#params of MLP model : {num_params_mlp}")
print(f"#params of CNN model : {num_params_cnn}")


## Training the model
Now we will train both MLP and CNN.

In [ ]:
learning_rate = 3e-3

mlp_optimizer = optim.SGD(mlp_model.parameters(), lr=learning_rate)
cnn_optimizer = optim.SGD(cnn_model.parameters(), lr=learning_rate)

total_epochs = 1

mlp_accuracy = train_model(mlp_model, mlp_optimizer, total_epochs)
cnn_accuracy = train_model(cnn_model, cnn_optimizer, total_epochs)

Q. Report the result. Which model performs better? Why do you think so?

A.

## Translational Equivariance

In this problem, we will check that CNN is translationally equivraint and MLP is not.

In [ ]:
batch_size = 4
stride_size = 4

## Some helpers
def torch_to_numpy(tensor):
    tensor = tensor.cpu().detach().numpy()
    return tensor

def preprocess_mnist_data(data):
    # padding tuples: (padding_left, padding_right, padding_top, padding_bottom)
    padded_data_list = []

    for i in range(0, 28, stride_size):
        for j in range(0, 28, stride_size):
            padded_data_list.append(F.pad(data, (j, 28-j, i, 28-i), mode='constant', value=0))
    
    padded_data = torch.stack(padded_data_list, dim=0)

    return padded_data

def preprocess_mnist_data_batch(samples, batch_size):
    result_list = []
    for i in range(batch_size):
        sample = T.functional.pil_to_tensor(samples[i][0])
        result_list.append(preprocess_mnist_data(sample))
    
    result = torch.stack(result_list, dim=0)
    return result

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(20, 20, 3, 1, padding=1)
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(20, 20, 3, 1, padding=1)
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(20, 20, 3, 1, padding=1)
        self.conv5 = nn.Conv2d(20, 100, 3, 1, padding=1)

    def forward(self, x):
        c1 = F.relu(self.conv1(x))
        c2 = F.relu(self.conv2(c1))
        p2 = self.maxpool2(c2)
        c3 = F.relu(self.conv3(p2))
        c4 = F.relu(self.conv4(c3))
        c5 = F.relu(self.conv5(c4))

        output = F.adaptive_avg_pool2d(c5, (1, 1))

        return (c1, c2, c3, c4, c5, output)

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(56*56, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)
        
    def forward(self, x):
        bs = x.shape[0]
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))   
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x)) 
        x = x.reshape((bs, 1, 10, 10))

        return x
        

### Generate the sample data

In [ ]:
mnist_train = dset.MNIST('./deeplearning/datasets', train=True, download=True)
mnist_samples = preprocess_mnist_data_batch(mnist_train, batch_size)

b, t, c, h, w = mnist_samples.shape
mnist_samples = mnist_samples.reshape((b*t, c, h, w))

In [ ]:
cnn_model = CNN().to(device)
mlp_model = MLP().to(device)

mnist_sample = mnist_samples.to(device)
# Convert to float32
mnist_sample = mnist_sample.float()

cnn_output = [torch_to_numpy(feat) for feat in cnn_model(mnist_sample)]
mlp_output = torch_to_numpy(mlp_model(mnist_sample))
mnist_sample = torch_to_numpy(mnist_sample)

# Reshaping features and input images
num_translations = (28//stride_size)**2
cnn_output = [feat.reshape((batch_size, num_translations, feat.shape[1], feat.shape[2], feat.shape[3])) for feat in cnn_output]
mlp_output = mlp_output.reshape((batch_size, num_translations, mlp_output.shape[1], mlp_output.shape[2], mlp_output.shape[3]))
mnist_sample = mnist_sample.reshape((batch_size, num_translations, mnist_sample.shape[1], mnist_sample.shape[2], mnist_sample.shape[3]))

### Visualize the output of MLP and CNN with different translations 
We visualize the output and intermediate features of CNN and the output of MLP with different translations. The output of CNN is invariant to translation. The intermediate feature changes as the input translates, but only as much as the input translation. However, the output of MLP is not invariant to translation. Note that MLP output and the CNN output don't have any spatial structure, while CNN intermediate features have. Therefore, we visualize the the spatial dimensions of intermediate features of CNN and the channel dimensions of outputs of MLP and CNN. This is the reason why both CNN and MLP output are not interpretable. However, CNN output is more invariant to translation than MLP output.

In [ ]:
fig = plt.figure(figsize=(5, 5))

# Main update function for interactive plot
def update_images(k):
    fig.clear()
    num_cnn_layers = len(cnn_output)
    
    f, axarr = plt.subplots(batch_size, num_cnn_layers+2, figsize=(15, 5))
    
    for i in range(batch_size):
        # Show the images
        axarr[i, 0].imshow(mnist_sample[i, k, :, :].squeeze())
        axarr[i, 0].set_title('Input Image')
        axarr[i, 0].axis('off')

        for j in range(0, num_cnn_layers):
            if j == num_cnn_layers - 1:
                feat_dim = cnn_output[j].shape[-1] # spatial dimension of the feature map
                cnn_feature = cnn_output[j][i, k, :].reshape((10, 10))
                axarr[i, j+1].set_title(f'CNN Output')
            else:
                cnn_feature = cnn_output[j][i, k, 0, :, :]
                axarr[i, j+1].set_title(f'CNN Layer {j+1}')
            axarr[i, j+1].imshow(cnn_feature)
            
            axarr[i, j+1].axis('off')

        axarr[i, -1].imshow(mlp_output[i, k, :, :].squeeze())
        axarr[i, -1].set_title(f'MLP Output')

# Create interactive plot
ip = interactive(update_images, k=widgets.IntSlider(min=0, max=48, step=1, value=0))
ip

Q. What do you observe? Why is it different from the case of MLP?

A.

Q. Note that even though CNN is not trained, the feature maps not only are still translationally equivariant but also extract a quite good features. Why is it so?

A.



### Goodness of CNN Features

As we can see the above example, even untrained CNN can extract very useful features for the complex image tasks. Now let's check how well the features extracted from the untrained CNN backbone can be used for the classification task. We will use the untrained CNN backbone and train only the final layer. Then we will check the performance of the final layer with the features extracted from the untrained CNN backbone. For more information about this, please refer to the paper [Deep Image Prior](https://dmitryulyanov.github.io/deep_image_prior).

In [ ]:
# Create a new model
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(10, 10, 3, 1, padding=1)
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(10, 10, 3, 1, padding=1)
        self.conv4 = nn.Conv2d(10, 10, 3, 1, padding=1)
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(10, 10, 3, 1, padding=1)
        self.conv6 = nn.Conv2d(10, 10, 3, 1, padding=1)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = x.flatten(start_dim=1)

        return x

class MNIST_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 100)

        
    def forward(self, x):
        bs = x.shape[0]
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))   
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x))

        return x

mlp_model = MNIST_MLP().to(device)
cnn_model = MNIST_CNN().to(device)

num_params_mlp = count_parameters(mlp_model)
num_params_cnn = count_parameters(cnn_model)

print(f"#params of MLP model : {num_params_mlp}")
print(f"#params of CNN model : {num_params_cnn}")

mnist_train = dset.MNIST('./deeplearning/datasets', train=True, download=True, transform=T.ToTensor())

# submnist_idx = (mnist_train.targets == 0) | (mnist_train.targets == 1)
# mnist_train.targets = mnist_train.targets[submnist_idx]
# mnist_train.data = mnist_train.data[submnist_idx]
mnist_loader = torch.utils.data.DataLoader(mnist_train, batch_size=256, shuffle=False)

cnn_features, mlp_features = [], []
label_list = []

for data, label in mnist_loader:
    bs = data.shape[0]
    data = data.to(device)
    label = label.to(device)
    
    cnn_output = cnn_model(data)
    mlp_output = mlp_model(data)

    if bs == 1:
        cnn_output = cnn_output.unsqueeze(0)

    cnn_features.append(torch_to_numpy(cnn_output))
    mlp_features.append(torch_to_numpy(mlp_output))

    label_list.append(torch_to_numpy(label))

cnn_features = np.concatenate(cnn_features, axis=0)
mlp_features = np.concatenate(mlp_features, axis=0)
label_list = np.concatenate(label_list, axis=0)

# sample 1000 images and labels randomly
sample_idx = np.random.choice(cnn_features.shape[0], 30000, replace=False)
cnn_features = cnn_features[sample_idx]
mlp_features = mlp_features[sample_idx]
label_list = label_list[sample_idx]

### Training the final layer

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(cnn_features, label_list, test_size=0.25, random_state=0)
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)
predictions = logisticRegr.predict(x_test)

# Use score method to get accuracy of model
score = logisticRegr.score(x_test, y_test)
print(score)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(mlp_features, label_list, test_size=0.25, random_state=0)
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)
predictions = logisticRegr.predict(x_test)

# Use score method to get accuracy of model
score = logisticRegr.score(x_test, y_test)
print(score)

Q. Explain the results. Why do you think the performance is better than the MLP?

A.